In [1]:
!nvidia-smi

Tue Mar 17 14:15:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 483 (delta 73), reused 114 (delta 28), pack-reused 311
Receiving objects: 100% (483/483), 75.90 MiB | 49.92 MiB/s, done.
Resolving deltas: 100% (180/180), done.


In [0]:
!pwd

/content


### ***PREPROCESSING***

In [0]:
import pandas as pd
import os
import shutil

In [14]:
df = pd.read_csv('/content/covid-chestxray-dataset/metadata.csv')
df.head()

,Patientid,offset,sex,age,finding,survival,view,date,location,filename,doi,url,license,clinical notes,other notes,Unnamed: 15
0,2,0.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
1,2,3.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
2,2,5.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
3,2,6.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
4,4,0.0,F,52.0,COVID-19,NaN,PA,2020,"Changhua Christian Hospital, Changhua City, Ta...",nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


In [0]:
df_covid = df[df['finding']=='COVID-19'] # we are concerned only with patients who are suffering from COVID-19

In [0]:
df_covid_PA = df_covid[df_covid['view']=='PA'] # we will only be using the PA view of the XRAY

In [17]:
df_covid_PA.head(10)

,Patientid,offset,sex,age,finding,survival,view,date,location,filename,doi,url,license,clinical notes,other notes,Unnamed: 15
0,2,0.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
1,2,3.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
2,2,5.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
3,2,6.0,M,65.0,COVID-19,Y,PA,2020,NaN,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,NaN,NaN,NaN
4,4,0.0,F,52.0,COVID-19,NaN,PA,2020,"Changhua Christian Hospital, Changhua City, Ta...",nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN
5,4,5.0,F,52.0,COVID-19,NaN,PA,2020,"Changhua Christian Hospital, Changhua City, Ta...",nejmc2001573_f1b.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive diffuse interstitial opacities and...,NaN,NaN
7,6,0.0,NaN,NaN,COVID-19,NaN,PA,2020,NaN,lancet-case2a.jpg,10.1016/S0140-6736(20)30211-7,https://www.thelancet.com/journals/lancet/arti...,NaN,Case 2: chest x-ray obtained on Jan 6 (2A). Th...,NaN,NaN
8,6,4.0,NaN,NaN,COVID-19,NaN,PA,2020,NaN,lancet-case2b.jpg,10.1016/S0140-6736(20)30211-7,https://www.thelancet.com/journals/lancet/arti...,NaN,Case 2: chest x-ray obtained on Jan 6 (2A). Th...,NaN,NaN
20,11,0.0,M,56.0,COVID-19,Y,PA,2020,Canada,1-s2.0-S0140673620303706-fx1_lrg.jpg,10.1016/S0140-6736(20)30370-6,https://www.sciencedirect.com/science/article/...,NaN,Chest x-ray shows bilateral peribronchovascula...,NaN,NaN
21,12,7.0,M,42.0,COVID-19,Y,PA,"January 1, 2020","Tongji Medical College, Wuhan, Hubei Province,...",nCoV-radiol.2020200269.fig1-day7.jpeg,10.1148/radiol.2020200269,https://pubs.rsna.org/doi/10.1148/radiol.20202...,NaN,"A, Chest radiograph obtained on day 7 after th...",NaN,NaN


In [18]:
df_covid_PA.index

Int64Index([  0,   1,   2,   3,   4,   5,   7,   8,  20,  21,  22,  24,  26,
             27,  29,  30,  31,  32,  33,  39,  40,  41,  42,  43,  48,  50,
             51,  53,  55,  70,  79,  80,  82,  83,  86,  88,  89,  90,  91,
             92,  93,  94,  95,  96,  97,  99, 101, 105, 108, 109, 113, 114,
            115, 116, 117, 118, 119, 122, 125],
           dtype='int64')

In [0]:
imagepath_covid = []
for i in df_covid_PA.index:
  imagepath_covid.append(df_covid_PA['filename'][i])

In [0]:
image_dataset_mixed = os.listdir('/content/covid-chestxray-dataset/images')

In [0]:
available_images_covid=list(set(imagepath_covid) & set(image_dataset_mixed))

In [0]:
!mkdir /content/image_dataset_covid/

In [0]:

for image in available_images_covid:
  src = '/content/covid-chestxray-dataset/images/'
  dst= '/content/image_dataset_covid/'
  shutil.copy(src+image, dst+image)


In [41]:
!unzip /content/normal.zip

Archive:  /content/normal.zip
  inflating: normal/IM-0033-0001-0001.jpeg  
  inflating: normal/IM-0240-0001.jpeg  
  inflating: normal/IM-0466-0001.jpeg  
  inflating: normal/NORMAL2-IM-0315-0001.jpeg  
  inflating: normal/NORMAL2-IM-0696-0001.jpeg  
  inflating: normal/NORMAL2-IM-0869-0001.jpeg  
  inflating: normal/NORMAL2-IM-1179-0001.jpeg  
  inflating: normal/person1_bacteria_2.jpeg  
  inflating: normal/person1102_bacteria_3043.jpeg  
  inflating: normal/person1290_virus_2215.jpeg  
  inflating: normal/person1558_bacteria_4066.jpeg  
  inflating: normal/person1599_virus_2776.jpeg  
  inflating: normal/person1830_bacteria_4693.jpeg  
  inflating: normal/person1935_bacteria_4849.jpeg  
  inflating: normal/person259_bacteria_1220.jpeg  
  inflating: normal/person339_bacteria_1574.jpeg  
  inflating: normal/person378_virus_761.jpeg  
  inflating: normal/person438_bacteria_1893.jpeg  
  inflating: normal/person525_bacteria_2216.jpeg  
  inflating: normal/person612_bacteria_2478.jpeg  

In [0]:
!mv /content/normal/* /content/dataset/normal 

In [0]:
!mv /content/image_dataset_covid/* /content/dataset/covid   #moving data of covid positive to datset directory

In [0]:
!rm -r  /content/dataset/covid/image_dataset_covid

### ***TRAINING***

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os


In [48]:
!python3 training_model.py -d /content/dataset/

[INFO] loading images...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-03-17 14:52:03.230579: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-17 14:52:03.256610: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-17 14:52:03.257219: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P4 major: 6 minor: 1 memoryClockRate(GHz): 1.1135
pciBusID: 0000:00:04.0
2020-03-17 14:52:03.257518: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-03-17 14:52:03.259030: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-03-17 14:52:03.260525: I tensorflow